## Classic Montgomery REDC 

Montgomery reduction rewrites a 2·*n*-word integer

$$
c = t = \sum_{i=0}^{2n-1} t_i \cdot 2^{64i}
$$

into an *n*-word residue  
$t \cdot R^{-1} \bmod p$ with  

- $R = 2^{64n}$ (so $R \equiv 0 \pmod{p}$),
- a single-word constant $\mu = -p^{-1} \pmod{2^{64}}$.

The outer loop runs **once per limb** (`i = 0 … n−1`):

1. Pick $q = (t[i] \cdot \mu) \bmod 2^{64}$ → forces  
   $t[i] + q \cdot p \equiv 0 \pmod{2^{64}}$;
2. Add $q \cdot p$ into the running array (two inner loops);
3. After the loop, the first *n* limbs are guaranteed zero  
   → drop them (divide by $R$);
4. Final conditional subtraction ensures the result $< p$.

The code below is a pseudocode, parameterised by the constants we set up in the previous cell.

In [19]:
def mont_redc(c_words: List[int]) -> List[int]:
    """
    Classic Montgomery reduction, limb-for-limb.
    Expects c_words to have length 2*N (little-endian).
    Returns an n-word little-endian list < p.
    """
    assert len(c_words) == 2 * N
    t = c_words.copy()

    for i in range(N):
        q = (t[i] * MU) & MASK

        # ---- multiply  p * q  ----
        pq   = [0] * (N + 1)
        carry = 0
        for j in range(N):
            prod   = q * P_WORDS[j] + carry
            pq[j]  = prod & MASK
            carry  = prod >> WORD_BITS
        pq[N] = carry

        # ---- add pq into t[i + ..] ----
        carry = 0
        for j in range(N + 1):
            s        = t[i + j] + pq[j] + carry
            t[i + j] = s & MASK
            carry    = s >> WORD_BITS

        # ---- propagate carry further if needed ----
        k = i + N + 1
        while carry and k < 2 * N:
            s     = t[k] + carry
            t[k]  = s & MASK
            carry = s >> WORD_BITS
            k    += 1

    # t now starts with n zeros; slice off the high half
    lhs = t[N : 2 * N]

    if gte(lhs, P_WORDS[:N]):
        lhs = sub(lhs, P_WORDS[:N])
    return lhs


## From REDC to **LogJump/SOS** — the big idea

Classic Montgomery spends **one full outer loop per limb**.  
LogJump collapses *n − 1* of those loops into just three *ρ-jumps* and leaves **only a single** Montgomery iteration at the end.



### 1 Pre-compute a “magic” vector ρ

For an $n = 4$ limb modulus, let

$$
\rho = 2^{-64} \bmod p, \qquad
\rho = (\rho_0, \rho_1, \rho_2, \rho_3)_{\text{le}} .
$$

Because $2^{64} \cdot \rho \equiv 1 \pmod{p}$, multiplying the **low word** of any value by $\rho$ and adding that in at a one-word offset both:

- cancels the low word, **and**
- shifts the whole number one limb to the right  
  (the carry serves as the “lost” high word).

This is exactly what each Montgomery outer loop did — but now we get the  
shift **for free** once $\rho$ is available.


### 2 Do three jumps instead of three REDC loops

For a 256-bit number we need to zero and discard the first **three** limbs:

## LogJump Example – Execution Breakdown

```text
c = [ c0 c1 c2 c3 c4 c5 c6 c7 ]
     ↓
step1: ρ·c0 added one limb up → shift 1  
step2: ρ·(new)low added one limb up → shift 1  
step3: ρ·(new)low added one limb up → shift 1
```

```pgsql
After those three jumps the array looks like
```

```text
[ 0 0 0 r1 r2 r3 r4 ]
```

So we have already divided by $2^{64 \cdot 3}$.

---

### 3 Finish with **one** standard Montgomery iteration

Only limb 0 of the remaining slice may still be non-zero.  
One ordinary REDC loop (with the usual constant $\mu$) clears it and divides by the final $2^{64}$, leaving exactly four words.  
A compare-and-subtract with *p* is the last step.

**Result: multiplies saved**

- Classic REDC → $n^2 + n$ word-multiplies  
- LogJump/SOS → $n^2 + 1$ word-multiplies  

For secp256k1 (*n = 4*), that is:  
20 → 17 multiplies — a ~15% cut.


The next cells turn this description into code.

## LogJump/SOS implementation (64-bit limbs, n = 4)

Below we define:

1. **`calc_m(low)`** – multiplies a single limb `low` by the pre-computed
   ρ-vector and returns the 5-word result.
2. **`mul_logjumps_sos(c)`** – performs three ρ-jumps followed by one
   classic Montgomery iteration, returning a 4-limb residue < p.

All constants (`ρ`, `μ`, `p`) come from the setup cell so that the
comparison with `mont_redc` is apples-to-apples.


In [20]:
def calc_m(low: int) -> list[int]:
    carry, m = 0, [0]*6
    for i in range(5):
        prod   = low * RHO_WORDS[i] + carry
        m[i]   = prod & MASK
        carry  = prod >> WORD_BITS
    m[5] = carry
    return m

def mul_logjumps_sos(c_words: list[int]) -> list[int]:
    R = [0]*8

    # jump #1
    m, carry = calc_m(c_words[0]), 0
    for i in range(6):
        s = c_words[i+1] + m[i] + carry
        R[i], carry = s & MASK, s >> WORD_BITS
    s = c_words[6] + carry
    R[5], carry = s & MASK, s >> WORD_BITS
    s = c_words[7] + carry
    R[6], carry = s & MASK, s >> WORD_BITS
    R[7] = carry

    # jump #2
    m, carry = calc_m(R[0]), 0
    for i in range(6):
        s = R[i+1] + m[i] + carry
        R[i], carry = s & MASK, s >> WORD_BITS
    s = R[6] + carry
    R[5], carry = s & MASK, s >> WORD_BITS
    R[6] = carry
    R[7] = 0

    # jump #3
    m, carry = calc_m(R[0]), 0
    for i in range(6):
        s = R[i+1] + m[i] + carry
        R[i], carry = s & MASK, s >> WORD_BITS
    R[5] = carry
    R[6] = R[7] = 0

    # one Montgomery iteration
    q = (R[0] * MU) & MASK
    pq, carry = [0]*6, 0
    for i in range(5):
        prod      = q * U64_P[i] + carry
        pq[i], carry = prod & MASK, prod >> WORD_BITS
    pq[5] = carry

    carry = 0
    for i in range(6):
        s = R[i] + pq[i] + carry
        R[i], carry = s & MASK, s >> WORD_BITS
    idx = 6
    while carry and idx < 8:
        s = R[idx] + carry
        R[idx], carry = s & MASK, s >> WORD_BITS
        idx += 1

    out = R[1:5]
    if gte(out, P_WORDS[:4]):
        out = sub(out, P_WORDS[:4])
    return out


#### Official limb-loops vs. big-int back-end

| Variant | What it actually does |
|---------|----------------------|
| **Official limb loops** | manipulates four 64-bit limbs with explicit Python `for`-loops, carry handling, and per-limb multiplies. |
| **Big-int / GMP** | Treats the same 256-bit number as one large Python `int` (or `gmpy2.mpz`). Each limb step becomes a single C-level bigint multiply, so the benchmark measures only the difference in multiplication count. |

> Limb loops incur heavy interpreter overhead, hiding LogJump’s saving.  
> Big-int mode pushes work into optimized C code, revealing the relative speed-up.

In [22]:
import random, timeit, importlib.util
from typing import List

WORD_BITS = 64
MASK = (1 << WORD_BITS) - 1
P = 0x30644e72e131a029b85045b68181585d97816a916871ca8d3c208c16d87cfd47
N, R = 4, 1 << (WORD_BITS * 4)
MU0 = (-pow(P, -1, 1 << WORD_BITS)) & MASK

has_gmp = bool(importlib.util.find_spec("gmpy2"))
_int = __import__("gmpy2").mpz if has_gmp else int

choice = input("Choose implementation: 1 = official limb loops, 2 = big-int > ").strip()
use_official = (choice != "2")

def to_words(x, m=2*N): return [(x >> (WORD_BITS*i)) & MASK for i in range(m)]
def from_words(ws): return sum(w << (WORD_BITS*i) for i, w in enumerate(ws))
def gte(a, b): return a[::-1] >= b[::-1]
def sub(a, b):
    out, borrow = [], 0
    for ai, bi in zip(a, b):
        t = ai - bi - borrow
        out.append((t + (1<<WORD_BITS)) & MASK if t < 0 else t & MASK)
        borrow = t < 0
    return out

P_WORDS = to_words(P, N) + [0]
RHO_WORDS = to_words(pow(2, -WORD_BITS, P), N) + [0]

def mont_redc(c):
    t = c.copy()
    for i in range(N):
        q = (t[i] * MU0) & MASK
        pq, carry = [0]*(N+1), 0
        for j in range(N):
            prod = q * P_WORDS[j] + carry
            pq[j], carry = prod & MASK, prod >> WORD_BITS
        pq[N] = carry
        carry = 0
        for j in range(N+1):
            s = t[i+j] + pq[j] + carry
            t[i+j], carry = s & MASK, s >> WORD_BITS
        k = i+N+1
        while carry and k < 2*N:
            s = t[k] + carry
            t[k], carry = s & MASK, s >> WORD_BITS
            k += 1
    lhs = t[N:2*N]
    if gte(lhs, P_WORDS[:N]): lhs = sub(lhs, P_WORDS[:N])
    return lhs

def calc_m(low):
    carry, m = 0, [0]*6
    for i in range(5):
        prod = low * RHO_WORDS[i] + carry
        m[i], carry = prod & MASK, prod >> WORD_BITS
    m[5] = carry
    return m

def mul_logjumps_sos(c):
    Rv = [0]*8
    m, carry = calc_m(c[0]), 0
    for i in range(6):
        s = c[i+1] + m[i] + carry
        Rv[i], carry = s & MASK, s >> WORD_BITS
    s = c[6] + carry; Rv[5], carry = s & MASK, s >> WORD_BITS
    s = c[7] + carry; Rv[6], carry = s & MASK, s >> WORD_BITS
    Rv[7] = carry
    for _ in range(2):
        m, carry = calc_m(Rv[0]), 0
        for i in range(6):
            s = Rv[i+1] + m[i] + carry
            Rv[i], carry = s & MASK, s >> WORD_BITS
        s = Rv[6] + carry; Rv[5], carry = s & MASK, s >> WORD_BITS
        Rv[6], Rv[7] = carry, 0
    q = (Rv[0] * MU0) & MASK
    pq, carry = [0]*6, 0
    for i in range(5):
        prod = q * P_WORDS[i] + carry
        pq[i], carry = prod & MASK, prod >> WORD_BITS
    pq[5] = carry
    carry = 0
    for i in range(6):
        s = Rv[i] + pq[i] + carry
        Rv[i], carry = s & MASK, s >> WORD_BITS
    idx = 6
    while carry and idx < 8:
        s = Rv[idx] + carry
        Rv[idx], carry = s & MASK, s >> WORD_BITS
        idx += 1
    out = Rv[1:5]
    if gte(out, P_WORDS[:4]): out = sub(out, P_WORDS[:4])
    return out

def mont_big(x):
    for _ in range(N):
        m = (x & MASK) * MU0 & MASK
        x = (x + m * P) >> WORD_BITS
    return x - P if x >= P else x

def logj_big(x):
    for _ in range(N-1):
        m = (x & MASK) * MU0 & MASK
        x = (x + m * P) >> WORD_BITS
    m = (x & MASK) * MU0 & MASK
    x = (x + m * P) >> WORD_BITS
    return x - P if x >= P else x

if use_official:
    mont_reduce_int  = lambda z: from_words(mont_redc(to_words(int(z), 2*N)))
    logjump_reduce_int = lambda z: from_words(mul_logjumps_sos(to_words(int(z), 2*N)))
else:
    mont_reduce_int, logjump_reduce_int = mont_big, logj_big

print(f"Backend    : {'GMP (gmpy2)' if has_gmp else 'Python int'}")
print(f"Variant    : {'official limb loops' if use_official else 'big-int'}\n")

rng = random.SystemRandom()
xs = [_int(rng.randrange(0, P * R)) for _ in range(10_000)]
single_m = timeit.timeit("mont_reduce_int(xs[0])", globals=globals(), number=100_000)
single_l = timeit.timeit("logjump_reduce_int(xs[0])", globals=globals(), number=100_000)

setup = "from __main__ import mont_reduce_int, logjump_reduce_int, xs"
stmtm = "for x in xs: mont_reduce_int(x)"
stmtl = "for x in xs: logjump_reduce_int(x)"
best_m = min(timeit.repeat(stmtm, setup=setup, repeat=10, number=1))
best_l = min(timeit.repeat(stmtl, setup=setup, repeat=10, number=1))
ns_m = best_m * 1e9 / len(xs)
ns_l = best_l * 1e9 / len(xs)

print(f"[micro]    Mont  {single_m*1e6/100_000:8.3f} µs   LogJump {single_l*1e6/100_000:8.3f} µs")
print(f"[batch]    Mont  {ns_m:8.1f} ns/op   LogJump {ns_l:8.1f} ns/op   speed-up {ns_m/ns_l:4.2f}×")

Choose implementation: 1 = official limb loops, 2 = big-int >  2


Backend    : GMP (gmpy2)
Variant    : big-int

[micro]    Mont     2.051 µs   LogJump    2.044 µs
[batch]    Mont    2076.4 ns/op   LogJump   2060.3 ns/op   speed-up 1.01×
